In [1]:
#Import Main Dependencies
import pandas as pd
from sqlalchemy import create_engine
from unicodedata import normalize
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [2]:
#Read in the training data file 
# https://drive.google.com/file/d/1r_nSU1w-Yu_yn6XKvUq8Wq7VC8CH9YN6/view?usp=share_link

train_df = pd.read_csv('/gdrive/My Drive/project4/Cathy/Resources/grocery_train.csv')
train_df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [3]:
len(train_df)

3000888

In [4]:
# check data types for SQL compatibility 
train_df.dtypes

id               int64
date            object
store_nbr        int64
family          object
sales          float64
onpromotion      int64
dtype: object

In [5]:
# convert 'date' column to datetime format for merging with other CSV's 
train_df['date'] = pd.to_datetime(train_df['date'])
train_df.dtypes

id                      int64
date           datetime64[ns]
store_nbr               int64
family                 object
sales                 float64
onpromotion             int64
dtype: object

In [6]:
# read in the holiday data file 
holiday_df = pd.read_csv('/gdrive/My Drive/project4/Cathy/Resources/holidays_events.csv')
holiday_df.tail()



,date,type,locale,locale_name,description,transferred
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False
349,2017-12-26,Additional,National,Ecuador,Navidad+1,False


In [7]:
len(holiday_df)

350

In [8]:
holiday_df.dtypes

date           object
type           object
locale         object
locale_name    object
description    object
transferred      bool
dtype: object

In [9]:
holiday_df['date'] = pd.to_datetime(holiday_df['date'])
holiday_df.dtypes

date           datetime64[ns]
type                   object
locale                 object
locale_name            object
description            object
transferred              bool
dtype: object

In [10]:
#drop rows that contain specific 'value' in 'column_name'
cleaned_hol = holiday_df[holiday_df['transferred'] == False]

In [11]:
# cleaned_holiday= holiday_df.loc[holiday_df["transferred"] != "FALSE"]
len(cleaned_hol)

338

In [12]:
cleaned_hol.drop_duplicates(subset='date', inplace=True)
len(cleaned_hol)

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


300

In [13]:
# drop transferred, cluster and type_x column 
holiday_df = cleaned_hol.drop(['type', "locale_name", "transferred" ], axis=1)
holiday_df.head()

,date,locale,description
0,2012-03-02,Local,Fundacion de Manta
1,2012-04-01,Regional,Provincializacion de Cotopaxi
2,2012-04-12,Local,Fundacion de Cuenca
3,2012-04-14,Local,Cantonizacion de Libertad
4,2012-04-21,Local,Cantonizacion de Riobamba


In [14]:
merged_holiday = pd.merge(train_df, holiday_df, on ='date', how="left")
merged_holiday.head()

,id,date,store_nbr,family,sales,onpromotion,locale,description
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,National,Primer dia del ano
1,1,2013-01-01,1,BABY CARE,0.0,0,National,Primer dia del ano
2,2,2013-01-01,1,BEAUTY,0.0,0,National,Primer dia del ano
3,3,2013-01-01,1,BEVERAGES,0.0,0,National,Primer dia del ano
4,4,2013-01-01,1,BOOKS,0.0,0,National,Primer dia del ano


In [15]:
len(merged_holiday)

3000888

In [16]:
# Read in the oil prices data file 
oil_df = pd.read_csv('/gdrive/My Drive/project4/Cathy/Resources/oil.csv')
oil_df.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [17]:
# drop null values from oil dataframe/csv
cleaned_oil=oil_df.dropna()
cleaned_oil.head()

,date,dcoilwtico
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
5,2013-01-08,93.21


In [18]:
cleaned_oil['date'] = pd.to_datetime(cleaned_oil['date'])
cleaned_oil.dtypes

<ipython-input-18-b014fd391d71>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_oil['date'] = pd.to_datetime(cleaned_oil['date'])


date          datetime64[ns]
dcoilwtico           float64
dtype: object

In [19]:
final_train = pd.merge(merged_holiday, cleaned_oil, on ='date', how="left")
final_train.head()

,id,date,store_nbr,family,sales,onpromotion,locale,description,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,National,Primer dia del ano,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,National,Primer dia del ano,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,National,Primer dia del ano,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,National,Primer dia del ano,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,National,Primer dia del ano,NaN


In [20]:
len(final_train)

3000888

In [21]:
final_train.tail()

,id,date,store_nbr,family,sales,onpromotion,locale,description,dcoilwtico
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,Local,Fundacion de Riobamba,47.57
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Local,Fundacion de Riobamba,47.57
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,Local,Fundacion de Riobamba,47.57
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Local,Fundacion de Riobamba,47.57
3000887,3000887,2017-08-15,9,SEAFOOD,16.000,0,Local,Fundacion de Riobamba,47.57


CATHY EDIT

In [22]:
# check volume of National, Local, Regional
print(final_train['locale'].value_counts())

National    229878
Local       176418
Regional     26730
Name: locale, dtype: int64


In [24]:
# df['Employer'] = df['Employer'].replace({'SELF': 'SELF-EMPLOYED', 'SELF EMPLOYED': 'SELF-EMPLOYED'})
final_train['locale'] = final_train['locale'].replace({'National': 1, 'Local': 0, 'Regional': 0})
final_train.head()

,id,date,store_nbr,family,sales,onpromotion,locale,description,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,1.0,Primer dia del ano,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,1.0,Primer dia del ano,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,1.0,Primer dia del ano,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,1.0,Primer dia del ano,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,1.0,Primer dia del ano,NaN


In [25]:
final_train_2 = final_train.dropna(subset=['locale'])
print(final_train_2)

              id       date  store_nbr                      family     sales  \
0              0 2013-01-01          1                  AUTOMOTIVE     0.000   
1              1 2013-01-01          1                   BABY CARE     0.000   
2              2 2013-01-01          1                      BEAUTY     0.000   
3              3 2013-01-01          1                   BEVERAGES     0.000   
4              4 2013-01-01          1                       BOOKS     0.000   
...          ...        ...        ...                         ...       ...   
3000883  3000883 2017-08-15          9                     POULTRY   438.133   
3000884  3000884 2017-08-15          9              PREPARED FOODS   154.553   
3000885  3000885 2017-08-15          9                     PRODUCE  2419.729   
3000886  3000886 2017-08-15          9  SCHOOL AND OFFICE SUPPLIES   121.000   
3000887  3000887 2017-08-15          9                     SEAFOOD    16.000   

         onpromotion  locale           

In [26]:
print(final_train_2['locale'].value_counts())

1.0    229878
0.0    203148
Name: locale, dtype: int64


In [28]:
final_train_2['locale'].isnull().values.any()

False

In [29]:
final_train_2.reset_index(drop=True)

,id,date,store_nbr,family,sales,onpromotion,locale,description,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,1.0,Primer dia del ano,NaN
1,1,2013-01-01,1,BABY CARE,0.000,0,1.0,Primer dia del ano,NaN
2,2,2013-01-01,1,BEAUTY,0.000,0,1.0,Primer dia del ano,NaN
3,3,2013-01-01,1,BEVERAGES,0.000,0,1.0,Primer dia del ano,NaN
4,4,2013-01-01,1,BOOKS,0.000,0,1.0,Primer dia del ano,NaN
...,...,...,...,...,...,...,...,...,...
433021,3000883,2017-08-15,9,POULTRY,438.133,0,0.0,Fundacion de Riobamba,47.57
433022,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,0.0,Fundacion de Riobamba,47.57
433023,3000885,2017-08-15,9,PRODUCE,2419.729,148,0.0,Fundacion de Riobamba,47.57
433024,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,0.0,Fundacion de Riobamba,47.57


In [31]:
final_train_2['locale'].isnull().sum()

0

In [42]:
# Use the Sklearn `train_test_split()` function to split the data into training and testing data

from sklearn.model_selection import train_test_split

### BEGIN SOLUTION

X = final_train_2['locale'].values.reshape(-1,1)
y = final_train_2['sales'].values.reshape(-1,1)

print(X.shape, y.shape)

(433026, 1) (433026, 1)


In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [44]:
# Create the model

### BEGIN SOLUTION

from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [45]:
# Fit the model
model.fit(X_train, y_train)

LinearRegression()

In [38]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()

# X = scaler.fit_transform(pd.DataFrame(X))
# Y = scaler.fit_transform(pd.DataFrame(y))

In [46]:
# # Fit the model to the training data. 

# ### BEGIN SOLUTION

# model.fit(X, Y)


In [47]:
# Calculate the mean_squared_error and the r-squared value
# for the testing data

from sklearn.metrics import mean_squared_error, r2_score

### BEGIN SOLUTION

# Use our model to make predictions
predicted = model.predict(X_test)

# Score the predictions with mse and r2
mse = mean_squared_error(y_test, predicted)
r2 = r2_score(y_test, predicted)

print(f"mean squared error (MSE): {mse}")
print(f"R-squared (R2): {r2}")

### END SOLUTION

mean squared error (MSE): 1592846.7794138745
R-squared (R2): 0.0004955063681998384


In [48]:
# Call the `score()` method on the model to show the R2 score

### BEGIN SOLUTION
model.score(X_test, y_test)
### END SOLUTION

0.0004955063681998384

In [ ]:
# #Read in the emissions data file 
# transactions_file = "resources/transactions.csv"
# trans_df = pd.read_csv(transactions_file)
# trans_df.head()

In [ ]:
# len(trans_df)

In [ ]:
# check data types for SQL compatibility 
# trans_df.dtypes

In [ ]:
# convert 'date' column to datetime format for merging with other CSV's 
# trans_df['date'] = pd.to_datetime(trans_df['date'])
# trans_df.dtypes

In [ ]:
# # we are focusing our machine learning models on thirty days from each year included 
# april_df = trans_df[((trans_df['date'] >= '2013-04-16') & (trans_df['date'] < '2013-05-16'))
#     | ((trans_df['date'] >= '2014-04-16') & (trans_df['date'] < '2014-05-16'))
#     | ((trans_df['date'] >= '2015-04-16') & (trans_df['date'] < '2015-05-16'))
#     | ((trans_df['date'] >= '2016-04-16') & (trans_df['date'] < '2016-05-16'))
#     | ((trans_df['date'] >= '2017-04-16') & (trans_df['date'] < '2017-05-16'))]
# april_df.head()

In [ ]:
# #checking to make sure the query cut off at desired date 
# april_df.tail()

In [ ]:
# # printing the number of rows in our datframe for quality assurance during merges 
# len(april_df)

In [ ]:
# #Read in the stores data file which has location info within Ecuador 
# stores_file = "resources/stores.csv"
# stores_df = pd.read_csv(stores_file)
# stores_df.head()

In [ ]:
# # checking that store number type is int64 prior to merge 
# stores_df.dtypes

In [ ]:
# # performing first merge 
# store_april = pd.merge(stores_df, april_df, on ='store_nbr')
# store_april.head()

In [ ]:
# #checking that number of rows is staying consistent 
# len(store_april)

In [ ]:
# # read in the holiday data file 
# holiday_file = "resources/holidays_events.csv"
# holiday_df = pd.read_csv(holiday_file)
# holiday_df.tail()

In [ ]:
# # checking length of this dataframe before cleaning it 
# len(holiday_df)

In [ ]:
# # since merge will take place on date column this datatype will need to be converted 
# holiday_df.dtypes

In [ ]:
# # performing conversion 
# holiday_df['date'] = pd.to_datetime(holiday_df['date'])
# holiday_df.dtypes

In [ ]:
# # excluding rows that cintain  official gvt observance info  
# cleaned_hol = holiday_df[holiday_df['transferred'] == False]

In [ ]:
# # checking that rows were removed
# len(cleaned_hol)

In [ ]:
# #removing duplicate rows before merging 
# cleaned_hol.drop_duplicates(subset='date', inplace=True)
# len(cleaned_hol)

In [ ]:
# # performing left join to prevent rows being added during merge 
# merged_holiday = pd.merge(store_april, cleaned_hol, on ='date', how="left")
# merged_holiday.head()

In [ ]:
# # confirming that number of rows is staying consistent 
# len(merged_holiday)

In [ ]:
# # Read in the oil prices data file 
# oil_file = "resources/oil.csv"
# oil_df = pd.read_csv(oil_file)
# oil_df.head()

In [ ]:
# # drop null values from oil dataframe/csv
# cleaned_oil=oil_df.dropna()
# cleaned_oil.head()

In [ ]:
# cleaned_oil['date'] = pd.to_datetime(cleaned_oil['date'])
# cleaned_oil.dtypes

In [ ]:
# # cleaned_hol.squeeze()
# indexed_oil = cleaned_oil.set_index('date', inplace=True)

In [ ]:
# cleaned_df = pd.concat([merged_holiday, indexed_oil], axis=1)
# len(cleaned_df)

In [ ]:
# # cleaned_df = pd.merge(trans_store_hol, cleaned_oil, on ='date')
# cleaned_df.head()

In [ ]:
# # drop transferred, cluster and type_x column 
# final_df = cleaned_df.drop(['type_x', "cluster", "transferred" ], axis=1)
# final_df.head()

In [ ]:
# final_df.dtypes

In [ ]:
# len(final_df)

In [ ]:
# #Create SQL Alchemy engine to postgres database "IMDB_db"
# protocol = 'postgresql'
# username = 'postgres'
# password = '0206Teddy'
# host = 'localhost'
# port = 5432
# database_name = 'grocery'
# rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
# engine = create_engine(rds_connection_string)

In [ ]:
# #show the existing table names in the database
# engine.table_names()

In [ ]:
# #write the data in the dataframe to the database table
# final_df.to_sql(name='earthquake_data', con=engine, if_exists='replace', index=False)

In [ ]:
# #Query the carbon emissions data table to confirm the above data load
# pd.read_sql_query('select * from public."earthquake_data"', con=engine)

In [ ]:
# pd.read_sql_query('select * from public."earthquake_data"', con=engine).count()